In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import pickle

2023-01-10 09:04:50.670654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 09:04:50.913225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-10 09:04:50.913266: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-10 09:04:52.591100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
MODEL_NO = 4
checkpoint_filepath = f'/media/ali/Drive/Static/NLP/twitter-sentiment/Models/{MODEL_NO}/best'

model = keras.models.load_model(f"{checkpoint_filepath}")

2023-01-10 09:04:54.457576: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-10 09:04:54.457645: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-10 09:04:54.457700: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-10 09:04:54.457752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-01-10 09:04:54.457805: W tensorfl

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 64)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 64, 64)            5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [4]:
# df = pd.read_csv("datasets/test2.csv", encoding = "ISO-8859-1")
# df = df.sample(frac=1)[:] # shuffle and truncate
# print(len(df.loc[df['sentiment']=="Neutral"]))
# print(len(df))
# df.drop(df.loc[df['sentiment']=="Neutral"].index, inplace=True)
# df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "Positive" else 0)

# print(len(df))

In [6]:
import sklearn

df = pd.read_csv("datasets/test.csv", encoding = "ISO-8859-1", names=['polarity', 'id', 'query', 'user', 'text'], index_col=2)
df = df.sample(frac=1)[:] # shuffle and truncate
df.drop(df.loc[df['polarity']==2].index, inplace=True)
print(set(df['polarity'].values))

map_val = {0:0, 4:2, 2:1}

real_labels = df['polarity'].map(lambda x: map_val[x])
df['polarity'] = df['polarity'].apply(lambda x: 1 if x == 4 else 0)

test_data = df['text'].to_numpy()
test_label = df['polarity'].to_numpy()

def map_vals(x):
    if x < 0.5:
        return 0
    elif x > 0.5:
        return 2
    return 1

predictions = (model.predict(test_data))
predictions = [map_vals(x) for x in predictions]

print(list(predictions - real_labels).count(0) / len(predictions))
print(sklearn.metrics.confusion_matrix(real_labels, predictions))

print(model.evaluate(test_data, test_label))


df = pd.read_csv("datasets/test2.csv", encoding = "ISO-8859-1")
df = df.sample(frac=1)[:] # shuffle and truncate
df.drop(df.loc[df['sentiment']=="Neutral"].index, inplace=True)
print(set(df['sentiment'].values))
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "Positive" else 0)


test_data = df['text'].to_numpy()
test_label = df['sentiment'].to_numpy()

predictions = (model.predict(test_data))
predictions = [map_vals(x) for x in predictions]

print(list(predictions - test_label).count(0) / len(predictions))
print(sklearn.metrics.confusion_matrix(real_labels, predictions))

print(model.evaluate(test_data, test_label))

1/1 [==============================] - 0s 30ms/step
[[-2.1220555]
 [-2.3641083]
 [ 1.163069 ]
 [-3.6090846]
 [ 2.3678331]
 [ 1.3543442]
 [ 3.5906782]
 [ 3.188873 ]
 [ 2.063805 ]
 [ 2.3505359]]
{0, 4}
12/12 [==============================] - 0s 16ms/step
0.8272980501392758
[[149  28]
 [ 34 148]]
12/12 [==============================] - 0s 14ms/step - loss: 0.4229 - accuracy: 0.8273
[0.42292842268943787, 0.8272980451583862]
{'Negative', 'Positive'}
336/336 [==============================] - 5s 14ms/step


ValueError: operands could not be broadcast together with shapes (10729,) (359,) 